In [1]:
import os
import pandas as pd
import numpy as np
import random
#Change params
NumLeadPeriods = 4
FGHJK = {1:"F", 2:"G", 3:"H", 4:"J", 5:"K", 6:"M", 7:"N", 8:"Q", 9:"U", 10:"V", 11:"X", 12:"Z"}

#Put all ZQX**.txt files under ./Data/zq

#Specify Stata location and edition
import stata_setup
stata_setup.config("C:\\Program Files\\Stata17\\", "se")
from pystata import stata


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       SE—Standard Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Unlimited-user , expiring 29 Jan 2022
Serial number: 401709308824
  Licensed to: Stata Developer
               StataCorp LLC

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. Maximum number of variables is set to 5,000; see help set_maxvar.


In [2]:
def graph_fomc_day_price(info_FF4_ticker, info_date, info_datetime):
    column_names = ["TimeStamp", "open", "high", "low", "close", "volume"]
    maindf  = pd.read_csv(os.path.join(os.getcwd(),"Data","zq","ZQ{}.txt".format(info_FF4_ticker)),                 
                     names=column_names, 
                     parse_dates=["TimeStamp"],
                     index_col=["TimeStamp"])
    fomc_window = maindf[info_date:info_date]
    fomc_window["datetime"] = fomc_window.index
    
    #Fill the gaps
    fomc_window = fomc_window.asfreq("1T", 'ffill')
    mask = fomc_window.index != fomc_window["datetime"]
    #Correct the error from auto interpolation
    fomc_window["volume"] = np.where(mask, 0, fomc_window["volume"])
    fomc_window["datetime"] = fomc_window.index
    
    stata.run("clear all")
    stata.pdataframe_to_data(fomc_window)
    stata.run("""gen double time = clock(datetime, "YMDhms")""")
    stata.run("""format time %tc""")
    stata.run("""tsset time""")
    stata.run("""local mytime = clock("{}", "YMDhms")""".format(info_datetime))
    stata.run("""
sum volume
local maxvol = r(max) * 10""")
    stata.run("""
    twoway (tsline close, tline(`mytime') title("{}") xlabel(`mytime') yaxis(1)) (tsline volume, yaxis(2) ylabel(minmax, axis(2)) yscale(axis(2) range(`maxvol')))""".format(info_date + " " + info_FF4_ticker))
    #stata.run("""graph export "{}-{}.png", as(png) name("Graph")""".format(os.path.join(os.getcwd(), info_date), info_FF4_ticker))
    
    return

In [2]:
def calc_price_movement(info_FF4_ticker, info_date, info_datetime):
    column_names = ["TimeStamp", "open", "high", "low", "close", "volume"]
    maindf  = pd.read_csv(os.path.join(os.getcwd(),"Data","zq","ZQ{}.txt".format(info_FF4_ticker)),                 
                     names=column_names, 
                     parse_dates=["TimeStamp"],
                     index_col=["TimeStamp"])
    fomc_window = maindf[info_date:info_date]
    
    #Fill the gaps
    fomc_window = fomc_window.asfreq("1T", 'ffill')
    #Correct the error from auto interpolation
    #fomc_window["datetime"] = fomc_window.index
    
    before = pd.Timestamp(info_datetime) - pd.Timedelta("10 minutes")
    after = pd.Timestamp(info_datetime) + pd.Timedelta("20 minutes")
    
    if len(fomc_window[before:before]) == 0:
        #This handling is problematic. Only 3 occurances. Assume problem away.
        before_val = fomc_window.first('1T')["close"][0]
    else:
        before_val = fomc_window[before:before]["close"][0]
    if len(fomc_window[after:after]) == 0:
        #By contrary, this handling is not problematic. 4 occurances.
        after_val = fomc_window.last('1T')["close"][0]
    else:
        after_val = fomc_window[after:after]["close"][0]
        
    return before_val - after_val 

In [3]:
fomc_dates = pd.read_csv(os.path.join(os.getcwd(),"Data","FOMC Dates.csv"),
                        parse_dates=["Next{}".format(i) for i in range(1, NumLeadPeriods+1)])
fomc_dates["Next0"] = pd.to_datetime(fomc_dates["Date"] + " " + fomc_dates["Time"])

results = []
for index, fomc_meeting in fomc_dates.iterrows():
    info_date = fomc_meeting["Next0"].strftime('%Y-%m-%d')
    info_time = fomc_meeting["Next0"].strftime('%H:%M')
    info_datetime = str(fomc_meeting["Next0"])
    
    print(info_date)
    result = [0]
    #Graph leads, lead 0 is just spot
    for lead in range(0, NumLeadPeriods+1):
        dlead = fomc_meeting["Next{}".format(lead)] #The date we are working with
        dim = dlead.days_in_month
        dd = dlead.day
        cutoff = int(0.75 * dim) #define a cut-off to circumvent div 0 problem. Implements Gurkeynak 2005.
        if dd >= cutoff:
            dleadlead = dlead + pd.offsets.MonthBegin(1)
            info_ticker = FGHJK[dleadlead.month] + str(dleadlead.year)[-2:]
        else:
            info_ticker = FGHJK[dlead.month] + str(dlead.year)[-2:]
        
        rdm = random.random()
        if rdm>0.95:
            #graph_fomc_day_price(info_ticker, info_date, info_datetime)
            pass

        print("Lead:", lead, info_ticker, round(calc_price_movement(info_ticker, info_date, info_datetime),4))
        price_movement = calc_price_movement(info_ticker, info_date, info_datetime)
        if dd >= cutoff:
            pass
        else:
            price_movement = (price_movement - ((dd / dim) * result[lead])) * (dim / (dim - dd))
        
        result.append(round(price_movement, 8))
        #calculate surprises
    result[0] = info_date
    results.append(result)
results

2009-01-28
Lead: 0 G09 0.005
Lead: 1 H09 0.005
Lead: 2 K09 0.0
Lead: 3 N09 -0.005
Lead: 4 Q09 -0.01
2009-03-18
Lead: 0 H09 -0.0025
Lead: 1 K09 -0.005
Lead: 2 N09 -0.02
Lead: 3 Q09 -0.025
Lead: 4 V09 -0.02
2009-04-29
Lead: 0 K09 0.0
Lead: 1 N09 -0.005
Lead: 2 Q09 0.0
Lead: 3 V09 0.01
Lead: 4 X09 0.01
2009-06-24
Lead: 0 N09 0.0
Lead: 1 Q09 0.005
Lead: 2 V09 0.005
Lead: 3 X09 0.01
Lead: 4 Z09 0.01
2009-08-12
Lead: 0 Q09 -0.0025
Lead: 1 V09 -0.005
Lead: 2 X09 -0.005
Lead: 3 Z09 -0.005
Lead: 4 G10 -0.015
2009-09-23
Lead: 0 V09 -0.01
Lead: 1 X09 -0.015
Lead: 2 Z09 -0.015
Lead: 3 G10 -0.035
Lead: 4 H10 -0.045
2009-11-04
Lead: 0 X09 -0.005
Lead: 1 Z09 0.0
Lead: 2 G10 -0.01
Lead: 3 H10 -0.015
Lead: 4 K10 -0.015
2009-12-16
Lead: 0 Z09 -0.005
Lead: 1 G10 -0.01
Lead: 2 H10 -0.01
Lead: 3 K10 0.005
Lead: 4 N10 0.01
2010-01-27
Lead: 0 G10 0.0
Lead: 1 H10 0.0
Lead: 2 K10 0.0
Lead: 3 N10 0.015
Lead: 4 Q10 0.02
2010-03-16
Lead: 0 H10 -0.0025
Lead: 1 K10 -0.01
Lead: 2 N10 -0.01
Lead: 3 Q10 -0.015
Lead: 4

Lead: 1 M19 -0.02
Lead: 2 Q19 -0.04
Lead: 3 U19 -0.04
Lead: 4 X19 -0.045
2019-06-19
Lead: 0 M19 0.025
Lead: 1 Q19 -0.035
Lead: 2 U19 -0.04
Lead: 3 X19 -0.055
Lead: 4 Z19 -0.065
2019-07-31
Lead: 0 Q19 0.0375
Lead: 1 U19 0.04
Lead: 2 X19 0.045
Lead: 3 Z19 0.045
Lead: 4 G20 0.04
2019-09-18
Lead: 0 U19 0.015
Lead: 1 X19 0.02
Lead: 2 Z19 0.025
Lead: 3 G20 0.015
Lead: 4 H20 0.02
2019-10-11
Lead: 0 V19 -0.005
Lead: 1 X19 -0.005
Lead: 2 Z19 -0.01
Lead: 3 G20 0.005
Lead: 4 H20 0.0
2019-10-30
Lead: 0 X19 -0.01
Lead: 1 Z19 0.005
Lead: 2 G20 0.01
Lead: 3 H20 0.015
Lead: 4 K20 0.015
2019-12-11
Lead: 0 Z19 0.0
Lead: 1 G20 0.0
Lead: 2 H20 0.0
Lead: 3 K20 0.0
Lead: 4 M20 -0.005
2020-01-29
Lead: 0 G20 0.015
Lead: 1 H20 0.01
Lead: 2 K20 0.005
Lead: 3 M20 0.005
Lead: 4 Q20 0.005


[['2009-01-28', 0.005, 0.005, 0.0, -0.005, -0.01315789],
 ['2009-03-18', -0.00596154, -0.005, -0.02, -0.02815789, -0.02],
 ['2009-04-29', 0.0, -0.005, 0.00315789, 0.01, 0.01],
 ['2009-06-24', 0.0, 0.00815789, 0.005, 0.01076923, 0.00917949],
 ['2009-08-12', -0.00407895, -0.005, -0.005, -0.005, -0.015],
 ['2009-09-23', -0.01, -0.01576923, -0.01417949, -0.035, -0.05566667],
 ['2009-11-04', -0.00576923, 0.00615385, -0.01, -0.02033333, -0.015],
 ['2009-12-16', -0.01033333, -0.01, -0.01, 0.005, 0.01],
 ['2010-01-27', 0.0, 0.0, 0.0, 0.015, 0.02238095],
 ['2010-03-16', -0.00516667, -0.01, -0.01, -0.01738095, -0.02611112],
 ['2010-04-28', 0.0, -0.01, -0.01738095, -0.00944445, -0.01006173],
 ['2010-05-09', 0.0, 0.0, -0.0147619, 0.0011111, 0.0054321],
 ['2010-06-23', 0.0, 0.0, 0.0, 0.0, 0.0],
 ['2010-08-10', 0.00369048, -0.00861112, 0.00095679, -0.00078794, -0.01],
 ['2010-09-21', 0.0, 0.00555556, 0.00454248, 0.0, 0.0],
 ['2010-11-03', 0.00555556, 0.00454248, 0.01, 0.01, 0.005],
 ['2010-12-14', 0

In [4]:
#Converted Monthly shocks
r = pd.DataFrame(results,
                columns=["FOMCDate", "Spot", "Lead1", "Lead2", "Lead3", "Lead4"])
r.index = r["FOMCDate"]
r = r.drop(labels = ["FOMCDate"], axis = 1)
r = r.drop(labels = ["2019-10-11"], axis = 0)
r.index = pd.to_datetime(r.index) + pd.offsets.MonthEnd(1)
r = r.asfreq("1M", fill_value=0)
r["Year"] = r.index.year
r["Month"] = r.index.month
r
r.to_csv("1-fomc-shocks.csv", columns = ["Year", "Month", "Spot", "Lead1", "Lead2", "Lead3", "Lead4"], index=False)
r.to_stata("1-fomc-shocks.dta", write_index = False)

In [5]:
#Raw daily shocks
r = pd.DataFrame(results,
                columns=["FOMCDate", "Spot", "Lead1", "Lead2", "Lead3", "Lead4"])
r.index = pd.to_datetime(r["FOMCDate"])
r["Year"] = r.index.year
r["Month"] = r.index.month
r["Day"] = r.index.day
r.to_stata("1-fomc-shocks-raw-daily.dta", write_index = False)